Project Report

Steven Hudler & Andrew Maybach

Introduction


The purpose of this project is to analyze how well-known different models perform on document classification. The models that will be used here are: Multinomial Bayes and Logistic Regression using Count Vectorizer and Tfidf Vectorizer. The dataset will be the twenty different topic based 1800 documents from sklearn's newsgroup. We will use the pandas dataframe, the nltk library, and sklearn's models with metrics. A variety of functions will be used through out this workbook.The first step was importing the necessary models and the fetch_20newsgroup from the sklearn.

In [1]:
# import pickle
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.corpus import names

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
from sklearn.svm import LinearSVC


/home/andy/Envs/text/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/andy/Envs/text/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/andy/Envs/text/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
training_data = fetch_20newsgroups(subset='train', shuffle=True,random_state=123) # added random state
testing_data = fetch_20newsgroups(subset='test', shuffle=True,random_state=123) # added random state

The newsgroup dataset are based on 20 topics from a sortment of different fields ranging from atheism to politics. The target/labels will be the 20 targets.

In [3]:
training_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Splitting the dataset into a test and train split. Shuffle is a necessary step to make sure the documents are evenly distributed between the test and train set.

In [4]:
type(training_data)

sklearn.utils.Bunch

After checking the data types of the train and test type. We transform the test and train into a pandas dataset.

In [5]:
training = pd.DataFrame({'Text': training_data.data, 'Label':training_data.target})
testing = pd.DataFrame({'Text': testing_data.data, 'Label':testing_data.target})

In [6]:
training.head()

,Text,Label
0,From: nextug@ac.dal.ca\nSubject: Powerbook 140...,4
1,From: kennejs@a.cs.okstate.edu (KENNEDY JAMES ...,16
2,From: robg@citr.uq.oz.au (Rob Geraghty)\nSubje...,13
3,From: eapu207@orion.oac.uci.edu (John Peter Ko...,1
4,From: neal@cmptrc.lonestar.org (Neal Howard)\n...,13


In [7]:
def clean_text(docs):
    '''This function lemmatizes texts and removes names'''
    out= []
    proper_names = names.words()
    lemma = WordNetLemmatizer()
    for words in docs.split():
        if words.isalpha() and words not in proper_names:
            out.append(lemma.lemmatize(words.lower())) 
        text = ' '.join(out)
    return text


Adding the cleaned text column to both training and testing dataframe.

In [8]:
training['cleaned_text'] = training['Text'].apply(lambda x: clean_text(x))
testing['cleaned_text'] = testing['Text'].apply(lambda x: clean_text(x))


Keeping only the cleaned text and labels in training and testing datasets.

In [9]:
training = training[['cleaned_text','Label']]
testing = testing[['cleaned_text','Label']]

In [10]:
# # not necessary but could be useful if the cleaned frame is needed elsewhere
# pickle.dump(training,open('training_frame.pickle','wb'))
# pickle.dump(testing,open('testing_frame.pickle','wb'))

Splitting the data to build a model

In [11]:
x_train,y_train = training['cleaned_text'],training['Label']
x_test,y_test = testing['cleaned_text'],testing['Label']

We previously experimented with different bag of words models and TFIDF performs consistently better than count vectorizer so for model testing we will be focusing on TFIDF. Below are the TFIDF parameters we will be testing on multiple models. The models are tested via pipeline and gridsearch over the aforementioned parameters. Gridsearch outputs a "Best Score" which is the best scoring on left out data. 

In [12]:
parms = {'vect__max_features' : (100,1000, 2000, 8000),
             'vect__max_df' : (0.25, 0.5),
             'vect__smooth_idf' : (True, False),
             'vect__sublinear_tf' : (True, False)}


In [13]:
pipeline1 = Pipeline([('vect',TfidfVectorizer(stop_words= 'english')),('clf',MultinomialNB())])
grid_search1 = grid_search = GridSearchCV(pipeline1, parms,cv = 3)
grid_search1.fit(x_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__max_features': (100, 1000, 2000, 8000), 'vect__max_df': (0.25, 0.5), 'vect__smooth_idf': (True, False), 'vect__sublinear_tf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
score1 = grid_search1.best_score_
score1

0.8331270991691709

In [17]:
pipeline2 = Pipeline([('vect',TfidfVectorizer(stop_words= 'english')),('clf',LogisticRegression())])
grid_search2 = grid_search = GridSearchCV(pipeline2, parms,cv = 3)
grid_search2.fit(x_train,y_train)

/home/andy/Envs/text/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/andy/Envs/text/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__max_features': (100, 1000, 2000, 8000), 'vect__max_df': (0.25, 0.5), 'vect__smooth_idf': (True, False), 'vect__sublinear_tf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
score2 = grid_search2.best_score_
score2

0.8469153261445996

In [22]:
pipeline3 = Pipeline([('vect',TfidfVectorizer(stop_words= 'english')),('clf',LinearSVC())])
grid_search3 = GridSearchCV(pipeline3, parms,cv = 3)
grid_search3.fit(x_train, y_train)



GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__max_features': (100, 1000, 2000, 8000), 'vect__max_df': (0.25, 0.5), 'vect__smooth_idf': (True, False), 'vect__sublinear_tf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [23]:
score3 = grid_search3.best_score_
score3

0.8713982676330211

Finally, we will test the data on our hold out test set

In [27]:
test1 = grid_search1.predict(x_test)
test2 = grid_search2.predict(x_test)
test3 = grid_search3.predict(x_test)

test_score1 = accuracy_score(test1,y_test)
test_score2 = accuracy_score(test2,y_test)
test_score3 = accuracy_score(test3,y_test)


Below are the best scores and test data scores from the Niave Bayes, Logistic and Linear Support Vector Machine

In [28]:
pd.DataFrame({'Model':['Multinomial Naive Bayes','Logistic Regression','Linear Support Vector Classifier'],
             'Best Scores': [score1,score2,score3],
             'Test Data Scores':[test_score1,test_score2,test_score3]})

,Model,Best Scores,Test Data Scores
0,Multinomial Naive Bayes,0.833127,0.771641
1,Logistic Regression,0.846915,0.774429
2,Linear Support Vector Classifier,0.871398,0.779076


As shown above our best model is the Linear Support Vector Classifier. Full fit details below.

In [30]:
grid_search3.best_params_

{'vect__max_df': 0.25,
 'vect__max_features': 8000,
 'vect__smooth_idf': False,
 'vect__sublinear_tf': True}